In [2]:
import numpy as np
import pandas as pd
import sqlite3 as sql
from sklearn.preprocessing import MinMaxScaler
from ipywidgets import interact # para análisis interactivo
from sklearn import neighbors # basado en contenido un solo producto consumido
import joblib

from surprise import Reader, Dataset
from surprise.model_selection import cross_validate, GridSearchCV
from surprise import KNNBasic, KNNWithMeans, KNNWithZScore, KNNBaseline
from surprise.model_selection import train_test_split

In [12]:
# conectar base de datos
conn=sql.connect('Data/movies2.db')
cur=conn.cursor()

# ver tablas disponibles en base de datos ###
cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
cur.fetchall()

[('ratings',),
 ('movies',),
 ('reco',),
 ('usuarios_sel',),
 ('movies_sel',),
 ('movies_m',),
 ('ratings_final',),
 ('full_ratings',),
 ('f_ratings',)]

In [13]:
df = pd.read_sql("SELECT * FROM f_ratings", conn)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45425 entries, 0 to 45424
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   user_id       45425 non-null  int64  
 1   movie_id      45425 non-null  int64  
 2   rating        45425 non-null  float64
 3   timestamp     45425 non-null  int64  
 4   movie_title   45425 non-null  object 
 5   movie_genres  45425 non-null  object 
 6   clean_title   45425 non-null  object 
 7   movie_year    45425 non-null  int64  
 8   fecha_nueva   45425 non-null  object 
dtypes: float64(1), int64(4), object(4)
memory usage: 3.1+ MB


In [14]:
df

,user_id,movie_id,rating,timestamp,movie_title,movie_genres,clean_title,movie_year,fecha_nueva
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Toy Story,1995,2000-07-30
1,1,3,4.0,964981247,Grumpier Old Men (1995),Comedy|Romance,Grumpier Old Men,1995,2000-07-30
2,1,6,4.0,964982224,Heat (1995),Action|Crime|Thriller,Heat,1995,2000-07-30
3,1,47,5.0,964983815,Seven (a.k.a. Se7en) (1995),Mystery|Thriller,Seven,0,2000-07-30
4,1,50,5.0,964982931,"Usual Suspects, The (1995)",Crime|Mystery|Thriller,"Usual Suspects, The",1995,2000-07-30
...,...,...,...,...,...,...,...,...,...
45420,609,589,3.0,847220990,Terminator 2: Judgment Day (1991),Action|Sci-Fi,Terminator 2: Judgment Day,1991,1996-11-05
45421,609,590,4.0,847220802,Dances with Wolves (1990),Adventure|Drama|Western,Dances with Wolves,1990,1996-11-05
45422,609,592,3.0,847220802,Batman (1989),Action|Crime|Thriller,Batman,1989,1996-11-05
45423,609,786,3.0,847221025,Eraser (1996),Action|Drama|Thriller,Eraser,1996,1996-11-05


# <b>4. Sistemas de recomendación basado en el contenido de cada usuario</b>
Este sistema se enfoca en analizar las interacciones pasadas de cada usuario con las películas para generar recomendaciones ajustadas a sus preferencias individuales. Utiliza el historial de visualización y las calificaciones otorgadas por el usuario para identificar patrones y gustos específicos. A partir de este análisis, el sistema sugiere películas que son más propensas a interesar al usuario, asegurando una experiencia de visualización más satisfactoria. 
La frecuencia de actualización de estas recomendaciones es mensual, lo que permite que el sistema tenga tiempo suficiente para detectar cambios en las preferencias del usuario. Este enfoque asegura que las recomendaciones sean relevantes y continúen mejorando a lo largo del tiempo, adaptándose a las nuevas elecciones y tendencias de visualización del usuario.

In [15]:
# Eliminar las columnas del DataFrame
movies = df.drop(columns=['user_id','rating', 'movie_title','fecha_nueva'])
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45425 entries, 0 to 45424
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   movie_id      45425 non-null  int64 
 1   timestamp     45425 non-null  int64 
 2   movie_genres  45425 non-null  object
 3   clean_title   45425 non-null  object
 4   movie_year    45425 non-null  int64 
dtypes: int64(3), object(2)
memory usage: 1.7+ MB


In [16]:
# Eliminar duplicados basados en la columna 'movie_clean_title'
movies = movies.drop_duplicates(subset=['clean_title'])
movies.reset_index(drop=True, inplace=True)

In [17]:
#### seleccionar usuario para recomendaciones ####
usuarios = pd.read_sql('select distinct (user_id) as user_id from ratings_final',conn)

In [18]:
# Configurar pandas para mostrar todas las columnas
pd.set_option('display.max_columns', None)

# Ver el DataFrame
print(movies)

     movie_id   timestamp                                 movie_genres  \
0           1   964982703  Adventure|Animation|Children|Comedy|Fantasy   
1           3   964981247                               Comedy|Romance   
2           6   964982224                        Action|Crime|Thriller   
3          47   964983815                             Mystery|Thriller   
4          50   964982931                       Crime|Mystery|Thriller   
..        ...         ...                                          ...   
852      6754  1113190465                        Action|Fantasy|Horror   
853      1172  1355184567                                        Drama   
854      8807  1464275372                             Adventure|Comedy   
855     45720  1464197299                                 Comedy|Drama   
856      4015  1084466683                                Comedy|Sci-Fi   

                             clean_title  movie_year  
0                              Toy Story        1995  
1

In [19]:
#### convertir a dummies
#### convertir a dummies

movies['clean_title'].nunique()
movies['movie_genres'].nunique()

col_dum=['clean_title','movie_genres']
dummies2=pd.get_dummies(movies,columns=col_dum)
dummies2.shape

(857, 1159)

In [20]:
def recomendar(user_id=list(usuarios['user_id'].value_counts().index)):

    ###seleccionar solo los ratings del usuario seleccionado
    ratings = pd.read_sql('select * from ratings_final where user_id=:user',conn, params={'user':user_id,})

    ###convertir ratings del usuario a array
    l_peliculas_s = ratings['movie_id'].to_numpy()

    ###agregar la columna de movieID de la pelicula a dummie para filtrar y mostrar nombre
    dummies2[['movie_id', 'clean_title']] = movies[['movie_id', 'clean_title']]

    ### filtrar libros calificados por el usuario
    peliculas_s = dummies2[dummies2['movie_id'].isin(l_peliculas_s)]

    ## eliminar columna nombre
    peliculas_s = peliculas_s.drop(columns=['movie_id','clean_title'])
    peliculas_s["indice"] = 1

    ##centroide o perfil del usuario
    centroide = peliculas_s.groupby("indice").mean()

    ### filtrar las peliculas no vistas
    peliculas_ns = dummies2[~dummies2['movie_id'].isin(l_peliculas_s)]

    ## eliminar clean_title y movie_id
    peliculas_ns = peliculas_ns.drop(columns=['movie_id','clean_title'])

    ### entrenar modelo
    model = neighbors.NearestNeighbors(n_neighbors=20, metric='cosine')
    model.fit(peliculas_ns)
    dist, idlist = model.kneighbors(centroide)
    ids=idlist[0] ### queda en un array anidado, para sacarlo
    recomend_movies = movies.loc[ids][['clean_title','movie_id']]
    vistas = movies[movies['movie_id'].isin(l_peliculas_s)][['clean_title','movie_id']]
    return recomend_movies

In [21]:
print(interact(recomendar))

interactive(children=(Dropdown(description='user_id', options=(1, 395, 411, 410, 409, 408, 405, 404, 403, 402,…

<function recomendar at 0x000001E800E67820>
